In [1]:
from itertools import product
from findiff.stencils import Stencil
offsets = list(product([-2,-1,0,1,2], repeat=2))
stencil = Stencil(offsets, partials={(1, 1): 1}, symbolic=True)
stencil

{(-2, -2): 1/144, (-2, -1): -1/18, (-2, 1): 1/18, (-2, 2): -1/144, (-1, -2): -1/18, (-1, -1): 4/9, (-1, 1): -4/9, (-1, 2): 1/18, (1, -2): 1/18, (1, -1): -4/9, (1, 1): 4/9, (1, 2): -1/18, (2, -2): -1/144, (2, -1): 1/18, (2, 1): -1/18, (2, 2): 1/144}

In [2]:
stencil.accuracy

4

This is how you can inspect what stencils `FinDiff` objects (implicitly) use:

In [3]:
from findiff import FinDiff
dxy = FinDiff((0, 1, 1), (1, 1, 1), acc=4)
stencil_set = dxy.stencil((10, 10))  # the shape argument is only used for determining the dimension of space, so any tuple of the right length will do
stencil_set

{('L', 'L'): {(4, 4): 0.06249999999999929, (4, 3): -0.3333333333333298, (4, 2): 0.749999999999993, (4, 1): -0.9999999999999922, (4, 0): 0.5208333333333297, (3, 4): -0.3333333333333298, (3, 3): 1.7777777777777604, (3, 2): -3.999999999999966, (3, 1): 5.333333333333296, (3, 0): -2.777777777777761, (2, 4): 0.749999999999993, (2, 3): -3.999999999999966, (2, 2): 8.999999999999934, (2, 1): -11.99999999999993, (2, 0): 6.249999999999969, (1, 4): -0.9999999999999922, (1, 3): 5.333333333333296, (1, 2): -11.99999999999993, (1, 1): 15.999999999999932, (1, 0): -8.333333333333306, (0, 4): 0.5208333333333297, (0, 3): -2.777777777777761, (0, 2): 6.249999999999969, (0, 1): -8.333333333333306, (0, 0): 4.340277777777767}, ('L', 'C'): {(4, 2): 0.020833333333333214, (4, 1): -0.1666666666666657, (4, -1): 0.1666666666666657, (4, -2): -0.020833333333333214, (3, 2): -0.11111111111111056, (3, 1): 0.8888888888888845, (3, -1): -0.8888888888888845, (3, -2): 0.11111111111111056, (2, 2): 0.24999999999999906, (2, 1): 

The stencil for the interior is the one for the `('C', 'C')` key where `C` stands for center of grid. The stencils at the boundaries at denoted by `L` and `H` characters where `L` means the boundary side where the index is low, and `H` where the index is high. So `('L', 'L')` is the 2D "lower left corner", `('C', 'L')` is the "left edge", etc.

So for the interior, we have

In [4]:
stencil_set.data[('C', 'C')]

{(2, 2): 0.006944444444444444,
 (2, 1): -0.05555555555555555,
 (2, -1): 0.05555555555555555,
 (2, -2): -0.006944444444444444,
 (1, 2): -0.05555555555555555,
 (1, 1): 0.4444444444444444,
 (1, -1): -0.4444444444444444,
 (1, -2): 0.05555555555555555,
 (-1, 2): 0.05555555555555555,
 (-1, 1): -0.4444444444444444,
 (-1, -1): 0.4444444444444444,
 (-1, -2): -0.05555555555555555,
 (-2, 2): -0.006944444444444444,
 (-2, 1): 0.05555555555555555,
 (-2, -1): -0.05555555555555555,
 (-2, -2): 0.006944444444444444}

Now let's get the offsets and construct symbolic stencils:

In [5]:
coefficients = {}
for key in stencil_set.data:
    offsets = list(stencil_set.data[key].keys())
    stc = Stencil(offsets, {(1, 1): 1}, spacings=[1, 1], symbolic=True)
    print(key, ': ', stc, '\n')

('L', 'L') :  {(4, 4): 1/16, (4, 3): -1/3, (4, 2): 3/4, (4, 1): -1, (4, 0): 25/48, (3, 4): -1/3, (3, 3): 16/9, (3, 2): -4, (3, 1): 16/3, (3, 0): -25/9, (2, 4): 3/4, (2, 3): -4, (2, 2): 9, (2, 1): -12, (2, 0): 25/4, (1, 4): -1, (1, 3): 16/3, (1, 2): -12, (1, 1): 16, (1, 0): -25/3, (0, 4): 25/48, (0, 3): -25/9, (0, 2): 25/4, (0, 1): -25/3, (0, 0): 625/144} 

('L', 'C') :  {(4, 2): 1/48, (4, 1): -1/6, (4, -1): 1/6, (4, -2): -1/48, (3, 2): -1/9, (3, 1): 8/9, (3, -1): -8/9, (3, -2): 1/9, (2, 2): 1/4, (2, 1): -2, (2, -1): 2, (2, -2): -1/4, (1, 2): -1/3, (1, 1): 8/3, (1, -1): -8/3, (1, -2): 1/3, (0, 2): 25/144, (0, 1): -25/18, (0, -1): 25/18, (0, -2): -25/144} 

('L', 'H') :  {(4, 0): -25/48, (4, -1): 1, (4, -2): -3/4, (4, -3): 1/3, (4, -4): -1/16, (3, 0): 25/9, (3, -1): -16/3, (3, -2): 4, (3, -3): -16/9, (3, -4): 1/3, (2, 0): -25/4, (2, -1): 12, (2, -2): -9, (2, -3): 4, (2, -4): -3/4, (1, 0): 25/3, (1, -1): -16, (1, -2): 12, (1, -3): -16/3, (1, -4): 1, (0, 0): -625/144, (0, -1): 25/3, (0, -2